### SocialAL Models
# Calc llh from general learning model for subset of data
KLS 12.19.22
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [ ]:
import sys
print(sys.version)  

### Load modules

In [ ]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import itertools
import os
import csv

## Pull in functions

In [ ]:
run common_functions.ipynb

In [ ]:
run single_alpha_functions.ipynb

### Pull in parameters fit to Trustworthy/Untrustworthy conditions

In [ ]:
pt = pd.read_csv('../../output/generalization_criterion/a1_fit2_2cond_part.csv')
pt.head(5)

### Load and clean data

In [ ]:
files = os.listdir('../../data/modeling')
files.remove('.DS_Store')
files.remove('missing')
#print(files)

In [ ]:
n_part = len(files)
print('number of participants: ', n_part)

In [ ]:
def load_and_clean(file):
    path = os.path.join('../../data/modeling', file)
    dt = pd.read_csv(path)
    # recode trial type into numbers for model
    def stims(trial_type):
        if trial_type == "Trustworthy":
            return 0
        elif trial_type == "Neutral":
            return 1
        elif trial_type == "Untrustworthy":
            return 2
    dt['Stim_Sequence'] = dt['trial_type'].apply(stims)
    # rename response_key to choice
    def choices(response_key):
        if response_key == 'None':
            return 0 
        else:
            return response_key  
    dt['Choice'] = dt['response_key'].apply(choices)
    dt['Choice'] = pd.to_numeric(dt['Choice'])
    # calculte the trustee response
    def resp(trial_earnings):
        if trial_earnings >= 12:
            return 1
        else:
            return 0
    dt['Trustee_Response'] = dt['trial_earnings'].apply(resp)
    data = dt[['Stim_Sequence','Choice', 'Trustee_Response']]
    # limit to neutral trials only
    data = data[data['Stim_Sequence'] == 1]
    data.reset_index(drop=True, inplace=True)
    return(data)

In [ ]:
# data = load_and_clean(files[0])
# data.head(20) # should only have 15 trials

In [ ]:
# create csv file
file = open('../../output/generalization_criterion/a1_llh4_1cond_part.csv', 'w')

# add header
headerList = ['id','a1_llh']
with open('../../output/generalization_criterion/a1_llh4_1cond_part.csv', 'a') as f_object: 
    dw = csv.DictWriter(f_object, delimiter = ',', fieldnames = headerList)
    dw.writeheader()

#loop thru participants
for x in range(n_part):
    print('loop:', x)
    subject = files[x][:-4]
    print(subject)
    
    # load and clean data
    print('loading data')
    data = load_and_clean(files[x])
    
    # pull parameters
    print('pull parameters')
    params = pt.loc[pt['id'] == subject].values.flatten().tolist()
    #print(params[1:4])
    
    # calculting llh
    print('calculating llh')
    llh = get_likelihood_action(params[1:4], data)
    a2 = [subject, llh]
    print(a2)
    
    # save results
    print('saving results')
    with open('../../output/generalization_criterion/a1_llh4_1cond_part.csv', 'a') as f_object: # create a file object
        writer_object = csv.writer(f_object) # pass the file object to csv writer
        writer_object.writerow(a2) #pass list as argument to writerow
        f_object.close() # close the file object